This project simulates how real cyber-security red-team professionals think.
Instead of hacking a system, we use multiple AI agents to:

Discover weaknesses (Recon Agent)

Suggest possible attacks (Attack Planner)

Calculate risk (Risk Agent)

Recommend defenses (Defense Agent)

It’s a safe, educational red-team environment — no real attacks are performed.

In [ ]:
!pip install -q gradio

In [ ]:
import os
import gradio as gr
from dataclasses import dataclass

Gemini API Key set up

Step-1: Go to Google AI Studio https://ai.google.dev/?utm_source=chatgpt.com

Step-2: Sign in with your Google account.

Step-3: Open the “API Keys” page. In the left sidebar, click on “API keys”.

Step-4: Create or select a project.

Step-5: Create a new API key.

Step-6: Copy and store the key safely.

In [ ]:
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI API KEY')# Insert your API key

Creating Data Structures

In [ ]:
@dataclass
class ReconResult:
    system_type: str
    findings: list
    notes: str

@dataclass
class AttackPlan:
    attack_paths: list
    focus_area: str

@dataclass
class RiskAssessment:
    overall_risk: str
    details: list

@dataclass
class DefensePlan:
    recommendations: list
    summary: str


ReconResult, AttackPlan, RiskAssessment, DefensePlan

Purpose:
We define the shapes of the outputs each agent will produce.

These act like containers that store each agent's findings.

Recon Agent
Reads your description and identifies security weaknesses based on keywords.

Example detections:

Input Description Agent Output No 2FA Weak authentication HTTP not HTTPS No encryption Shared admin Insider threat

Attack Planner Agent
Suggests theoretical attacks an attacker could try, based on recon findings.

Risk Agent
Assigns a risk level: LOW MEDIUM HIGH CRITICAL

Defense Agent
Suggests security fixes based on weaknesses.

Example: Weakness Defense No HTTPS Enable TLS certificates Old PHP Upgrade software

Report Generator
Combines the outputs of all agents into a professional-style red-team report.

In [ ]:
def recon_agent(system_type: str, description: str) -> ReconResult:
    desc = description.lower()
    findings = []

    if "no 2fa" in desc:
        findings.append("Missing multi-factor authentication (MFA/2FA).")
    if "http" in desc and "https" not in desc:
        findings.append("Unencrypted HTTP traffic (HTTPS not enforced).")
    if "outdated" in desc or "legacy" in desc:
        findings.append("Outdated / legacy software present.")
    if "weak password" in desc or "admin" in desc:
        findings.append("Weak or default credentials detected.")
    if "shared" in desc:
        findings.append("Shared account / shared credentials risk.")
    if "no logging" in desc:
        findings.append("Lack of centralized logging and monitoring.")
    if "public" in desc and "bucket" in desc:
        findings.append("Public cloud storage buckets exposed.")

    if not findings:
        findings.append("No obvious weaknesses identified.")

    notes = "Recon simulated based only on provided text. No real scanning performed."
    return ReconResult(system_type, findings, notes)


def attack_planner_agent(recon: ReconResult, focus: str) -> AttackPlan:
    attack_paths = []
    for f in recon.findings:
        t = f.lower()
        if "mfa" in t:
            attack_paths.append("Credential attacks become viable due to lack of MFA.")
        if "http" in t:
            attack_paths.append("MITM attack possible due to lack of HTTPS.")
        if "legacy" in t:
            attack_paths.append("Public CVEs may apply to outdated software.")
        if "weak" in t:
            attack_paths.append("Brute-force or password spray likely to succeed.")
    if not attack_paths:
        attack_paths.append("No specific attack path identified.")
    return AttackPlan(attack_paths, focus)


def risk_agent(recon: ReconResult, plan: AttackPlan) -> RiskAssessment:
    score = len(recon.findings)
    if score >= 5: level = "CRITICAL"
    elif score >= 3: level = "HIGH"
    elif score == 2: level = "MEDIUM"
    else: level = "LOW"
    details = [
        f"Detected {len(recon.findings)} issue(s).",
        f"Overall simulated risk rating: {level}"
    ]
    return RiskAssessment(level, details)


def defense_agent(recon: ReconResult, risk: RiskAssessment) -> DefensePlan:
    recs = []
    for f in recon.findings:
        t = f.lower()
        if "mfa" in t: recs.append("Implement MFA for all logins.")
        if "http" in t: recs.append("Enable HTTPS.")
        if "legacy" in t: recs.append("Upgrade outdated frameworks.")
        if "weak" in t: recs.append("Enforce password policies.")
        if "logging" in t: recs.append("Add centralized logging.")

    if not recs:
        recs.append("No specific recommendations.")

    summary = f"System risk level is {risk.overall_risk}. Apply recommended fixes in order of priority."
    return DefensePlan(recs, summary)


def generate_report(recon, plan, risk, defense):
    return f"""
## 🛡 Autonomous Multi-Agent Red Team Report

### 1️⃣ System Type
{recon.system_type}

### 2️⃣ Recon Findings
{chr(10).join('- ' + f for f in recon.findings)}

### 3️⃣ Attack Paths
{chr(10).join('- ' + p for p in plan.attack_paths)}

### 4️⃣ Risk Assessment
{chr(10).join('- ' + d for d in risk.details)}

### 5️⃣ Defense Recommendations
{chr(10).join('- ' + r for r in defense.recommendations)}

**Summary:** {defense.summary}
"""


def run_red_team(system_type, description, focus_area):
    if not description.strip():
        return "❌ Please enter a system description."
    recon = recon_agent(system_type, description)
    plan = attack_planner_agent(recon, focus_area)
    risk = risk_agent(recon, plan)
    defense = defense_agent(recon, risk)
    return generate_report(recon, plan, risk, defense)

# Gradio

Enables building interactive web interfaces for Python applications without any frontend development.

Lets users enter inputs, run the model, and view outputs directly in the browser with ease.

In [ ]:
with gr.Blocks(title="Multi-Agent Red Team") as demo:
    gr.Markdown("## 🤖 Autonomous Multi-Agent Red Team (Simulation Mode)")
    system_type = gr.Dropdown(
        ["Web Application", "Mobile App", "Internal Network", "Cloud Application"],
        label="System Type")
    description = gr.Textbox(label="Describe system & weaknesses", lines=5)
    focus = gr.Dropdown(["General", "Authentication", "Network Exposure"], label="Attack Focus")
    output = gr.Markdown()
    gr.Button("Run Red-Team Simulation 🚀").click(run_red_team, [system_type, description, focus], output)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://853d084e39116657df.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
